# Colbert

In [ ]:
!git -C ColBERT/ pull || git clone https://github.com/stanford-futuredata/ColBERT.git
import sys; sys.path.insert(0, 'ColBERT/')

Already up to date.


## Si on run en dehors de collab regarder le readme (explique comment import sur conda)

In [ ]:
try:
    import google.colab
    !pip install -U pip
    !pip install -e ColBERT/
    !pip install faiss-gpu torch
except Exception:
  import sys; sys.path.insert(0, 'ColBERT/')
  try:
    from colbert import Indexer, Searcher
  except Exception:
    print("If you're running outside Colab, please make sure you install ColBERT in conda following the instructions in our README. You can also install (as above) with pip but it may install slower or less stable faiss or torch dependencies. Conda is recommended.")
    assert False

Obtaining file:///content/ColBERT
  Preparing metadata (setup.py) ... done
  Attempting uninstall: colbert-ai
    Found existing installation: colbert-ai 0.2.20
    Uninstalling colbert-ai-0.2.20:
      Successfully uninstalled colbert-ai-0.2.20
  DEPRECATION: Legacy editable install of colbert-ai==0.2.20 from file:///content/ColBERT (setup.py develop) is deprecated. pip 25.0 will enforce this behaviour change. A possible replacement is to add a pyproject.toml or enable --use-pep517, and use setuptools >= 64. If the resulting installation is not behaving as expected, try using --config-settings editable_mode=compat. Please consult the setuptools documentation for more information. Discussion can be found at https://github.com/pypa/pip/issues/11457
  Running setup.py develop for colbert-ai


In [ ]:
import colbert

In [ ]:
from colbert import Indexer, Searcher
from colbert.infra import Run, RunConfig, ColBERTConfig
from colbert.data import Queries, Collection

In [ ]:
import pandas as pd

file_path = "/content/wiki_movie_plots_deduped.csv"
df = pd.read_csv(file_path)

# Supprimer les lignes avec des valeurs manquantes
df = df.dropna(subset=['Plot', 'Title'])

collection = df['Plot'].tolist()
titles = df['Title'].tolist()

In [ ]:
# Configuration pour l'indexation
checkpoint = 'colbert-ir/colbertv2.0'
index_name = 'wikipedia_movie_plots'
doc_maxlen = 300                            # Longueur maximale des documents
nbits = 2                                   # Encodage en 2 bits pour la compression

# Indexer les documents
#   nranks=1  :  nombre de machine sur laquel on execute
#   experiment='movie_plots' : nom de l'expérience
with Run().context(RunConfig(nranks=1, experiment='movie_plots')):

    # Configuration Colbert (kmeans_niters = nb itération algo kmean)
    # Kmeans est utilisé pour regrouper les vecteurs en cluster => réduire la taille du stockage (car on stock que les centroides des cluster) et + rapide car on compare juste la distance entre centroïde et requète
    config = ColBERTConfig(doc_maxlen=doc_maxlen, nbits=nbits, kmeans_niters=4)

    # Initialisation indexuer
    indexer = Indexer(checkpoint=checkpoint, config=config)

    # Indexer la collection
    indexer.index(name=index_name, collection=collection, overwrite=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


artifact.metadata:   0%|          | 0.00/1.63k [00:00<?, ?B/s]



[Jan 03, 08:02:03] #> Creating directory /content/experiments/movie_plots/indexes/wikipedia_movie_plots 


#> Starting...
#> Joined...


In [ ]:
from colbert import Searcher

nb_result = 3

# Charge l'index dense et configure le searcher
with Run().context(RunConfig(experiment='movie_plots')):
    searcher = Searcher(index=index_name, collection=collection)

query = "A movie about space exploration and human survival"
print(f"Requete: {query}")

results = searcher.search(query, k=nb_result)
print(f"Résultats: {results}")

id = results[0]
rank = results[1]
score = results[2]

for i in range(nb_result):
    print(f"[Rank {rank[i]}] (Score: {score[i]:.2f})")
    print(f"Title: {titles[id[i]]}")
    print(f"Plot: {collection[id[i]]}\n")


[Jan 03, 08:58:52] #> Loading codec...
[Jan 03, 08:58:52] #> Loading IVF...
[Jan 03, 08:58:52] #> Loading doclens...


/content/ColBERT/colbert/utils/amp.py:12: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler()
/content/ColBERT/colbert/indexing/codecs/residual.py:141: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use c

[Jan 03, 08:58:52] #> Loading codes and residuals...



  0%|          | 0/2 [00:00<?, ?it/s]/content/ColBERT/colbert/indexing/codecs/residual_embeddings.py:86: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(cod

Requete: A movie about space exploration and human survival

#> QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
#> Input: A movie about space exploration and human survival, 		 True, 		 None
#> Output IDs: torch.Size([32]), tensor([ 101,    1, 1037, 3185, 2055, 2686, 8993, 1998, 2529, 7691,  102,  103,
         103,  103,  103,  103,  103,  103,  103,  103,  103,  103,  103,  103,
         103,  103,  103,  103,  103,  103,  103,  103], device='cuda:0')
#> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')

Résultats: ([8167, 16430, 22892], [1, 2, 3], [20.671875, 20.171875, 19.640625])
[Rank 1] (Score: 20.67)
Title: Women of the Prehistoric Planet
Plot: A spacefaring crew from an advanced civilization is preparing to return home after an extended voyage. The crew includes "humans" (represented in the film by Caucasian actors and actresses) and "Centaurian